<a href="https://colab.research.google.com/github/samantamrityunjay/ICDcoding/blob/master/DataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from ast import literal_eval
import re

Code to mount google drive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


<h3> Tables 'adm_icd' and 'diagnoses_icd'
<p>'adm_icd' contains all the Admission Ids of the MIMIC database. <br>
There are total <strong><u>58976</u></strong> admisssion ids.</p>
 <p>Out of which <strong><u>47</u></strong>  do not have any ICD9 codes corresponding to them (NaN values). So we have total of <strong><u>58929</u></strong> in this table with correponding ICD9 codes and converted ICD10 codes.
 <p>There are <strong><u>6984</u></strong>unique ICD9 codes and <strong><u>942</u></strong> ICD9 categories.

In [3]:
adm_icd=pd.read_csv('/content/drive/My Drive/AlgoIntern/Data/Data1.0/adm_icd.csv')

In [4]:
adm_icd

,HADM_ID,ICD9_CODE,ICD10
0,100001,"25013,3371,5849,5780,V5867,25063,5363,4580,250...","Misc,Misc,N179,Misc,Z794,Misc,Misc,Misc,Misc,I..."
1,100003,"53100,2851,07054,5715,5715,5715,45621,53789,40...","Misc,D62,B182,K740,K7460,K7469,Misc,Misc,I10,M..."
2,100006,"49320,51881,51881,486,20300,2761,7850,3090,V12...","Misc,J9600,J9690,J189,Misc,E871,Misc,Misc,Z867..."
3,100007,"56081,5570,9973,486,4019","Misc,Misc,Misc,J189,I10"
4,100009,"41401,99604,4142,25000,27800,V8535,4148,4111,V...","I2510,Misc,Misc,E119,E669,Misc,Misc,I200,Z9861..."
...,...,...,...
58924,199993,"41031,42821,42731,4271,5180,5180,4240,4240,276...","Misc,Misc,I4891,I472,J9811,J9819,I340,I348,E87..."
58925,199994,"486,4280,51881,51881,3970,496,4169,585,42732,2762","J189,I509,J9600,J9690,Misc,J449,Misc,Misc,I489..."
58926,199995,"4210,7464,42971,30401,4412,44284,V1259,04111,3...","Misc,Misc,Misc,Misc,Misc,Misc,Misc,Misc,Misc,F..."
58927,199998,"41401,9971,9971,9971,9971,9975,42731,78820,411...","I2510,I97710,I97790,I9788,I9789,Misc,I4891,Mis..."


In [5]:
allmimic=pd.read_csv('/content/drive/My Drive/AlgoIntern/Data/DIAGNOSES_ICD.csv')

In [6]:
allmimic

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275


In [7]:
allmimic.HADM_ID.nunique()

58976

In [8]:
admID_without_ICD9codes=allmimic.loc[allmimic.ICD9_CODE.isna()].HADM_ID.to_list()

In [9]:
len(admID_without_ICD9codes)

47

In [10]:
allmimic.ICD9_CODE.nunique()

6984

In [11]:
def category(x):
  if type(x)==str:
    return re.split("(\d{3}|[VE]\d{2})",x)[1]
  else:
    return None

In [12]:
allmimic['ICD9_CATEGORY']=allmimic.ICD9_CODE.apply(category)

In [13]:
print(allmimic.ICD9_CATEGORY.nunique())

942


In [14]:
allmimic.loc[allmimic.ICD9_CATEGORY.isna()].nunique()

ROW_ID           47
SUBJECT_ID       47
HADM_ID          47
SEQ_NUM           0
ICD9_CODE         0
ICD9_CATEGORY     0
dtype: int64

>'notes_icd' contains merged tables of 'adm_ids' and 'discharge summarry table'
>> There are total <strong><u>52726</u></strong><sup>*</sup> admission ids in discharge summary, with corresponding <strong><u>6919</u></strong> ICD9 codes and <strong><u>941</u></strong> ICD9 categories.<br>
<sup>*</sup>4 (actually 6 but 4 are unique) do not have any corresponding ICD codes. These are part of 47 discussed above

In [15]:
notes_icd=pd.read_csv('/content/drive/My Drive/AlgoIntern/Data/Data1.0/notes_icd.csv')

In [16]:
notes_icd

,SUBJECT_ID,HADM_ID,CATEGORY,DESCRIPTION,ISERROR,TEXT,text_mod,sec,split,service:,social history:,family history:,past medical history:,chief complaint:,discharge medications:,discharge disposition:,discharge diagnosis:,allergies:,physical exam:,ICD9_CODE_x,ICD10,ICD9_CODE_y,icd10_pcs
0,22532,167853.0,Discharge summary,Report,NaN,Admission Date: [**2151-7-16**] Dischar...,admission date: <date> discharge date...,"['Admission Date:', 'Discharge Date:', 'Servic...",['service: addendum: radiologic studies: ...,service: addendum: radiologic studies: ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"01193,4254,4254,42731,2639,2762,5070,5119,2113","Misc,I425,I428,I4891,E46,E872,J690,J918,Misc","4542,3491,3491,3893,3891","Misc,0W9.93ZZ,0W9.B3ZZ,Misc,Misc"
1,13702,107527.0,Discharge summary,Report,NaN,Admission Date: [**2118-6-2**] Discharg...,admission date: <date> discharge date...,"['Admission Date:', 'Discharge Date:', 'Birth:...",[' service: micu and then to <name> medicine ...,service: micu and then to <name> medicine ...,"social history: lives with her husband, dr. ...",family history: noncontributory.,past medical history: 1. copd. last pulmona...,NaN,discharge medications: 1. levothyroxine 75 mc...,NaN,NaN,allergies: norvasc leads to lightheadedness ...,NaN,"5191,49121,51881,51881,486,2761,2449,311","Misc,Misc,J9600,J9690,J189,E871,E039,F329","9390,9390,9390,966,966,966,3199,9671,9671,9604...","5A0.9357,5A0.9457,5A0.9557,0DH.67UZ,0DH.68UZ,3..."
2,13702,167118.0,Discharge summary,Report,NaN,Admission Date: [**2119-5-4**] D...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Servic...","[' service: cardiothoracic ', ' social hist...",service: cardiothoracic,social history: social history: the patient ...,family history: family history: (+) fhx cad;...,past medical history: copd flare [**6-7**] s...,"chief complaint: 81 yo f smoker w/ copd, sev...",discharge medications: 1. albuterol sulfate ...,discharge disposition: extended care faci...,"discharge diagnosis: copd, coronary artery di...",allergies: amlodipine attending:<name>,physical exam: admit h+p general-lovely 81 ...,"5191,5185,496,2762,45340,5533","Misc,Misc,J449,E872,Misc,Misc","3179,311,9672,9605,9605,9605,9605,9605,9605,96...","Misc,Misc,5A1.955Z,0B7.17DZ,0B7.18DZ,0BH.07DZ,..."
3,13702,196489.0,Discharge summary,Report,NaN,Admission Date: [**2124-7-21**] ...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Servic...","[' service: medicine ', ' social history: ...",service: medicine,social history: the patient is married and w...,family history: (+) fhx cad; father with an ...,past medical history: # copd flare fev1 40% ...,chief complaint: copd exacerbation/shortness...,discharge medications: 1. metoprolol succina...,discharge disposition: extended care faci...,discharge diagnosis: primary: 1. chronic obs...,allergies: amlodipine attending:<name>,physical exam: on admission vitals: t: bp: ...,"51884,5849,34830,49121,2760,4160,3594,5780,276...","Misc,N179,Misc,Misc,E870,Misc,Misc,Misc,E872,M...","9672,9604,9604,3891,3893,4513,966,966,966","5A1.955Z,0BH.17EZ,0BH.18EZ,Misc,Misc,0DJ.08ZZ,..."
4,26880,135453.0,Discharge summary,Report,NaN,Admission Date: [**2162-3-3**] D...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: medicine ', ' social history: ...",service: medicine,social history: patient recently discharged ...,family history: non contributory,past medical history: coronary artery diseas...,chief complaint: mr. <name> was seen at <hos...,discharge medications: 1. bisacodyl 10 mg su...,discharge disposition: extended care faci...,discharge diagnosis: 1. cervical spondylosis ...,allergies: patient recorded as having no kno...,physical exam: phycial exam prior to surgery...,"80506,5070,42823,2930,4538,E882,4280,4011,V450...","Misc,J690,Misc,F05,Misc,Misc,I509,Misc,Misc,I4...","8102,8103,353,8163,9671,9671,9604,9604","Misc,Misc,Misc,Misc,5A1.935Z,5A1.945Z,0BH.17EZ..."
...,.

In [17]:
notes_icd.CATEGORY.unique()

array(['Discharge summary'], dtype=object)

In [18]:
notes_icd.HADM_ID.nunique()

52726

In [19]:
notes_icd.loc[notes_icd.HADM_ID.isna()].count()

SUBJECT_ID                0
HADM_ID                   0
CATEGORY                  0
DESCRIPTION               0
ISERROR                   0
TEXT                      0
text_mod                  0
sec                       0
split                     0
service:                  0
social history:           0
family history:           0
past medical history:     0
chief complaint:          0
discharge medications:    0
discharge disposition:    0
discharge diagnosis:      0
allergies:                0
physical exam:            0
ICD9_CODE_x               0
ICD10                     0
ICD9_CODE_y               0
icd10_pcs                 0
dtype: int64

In [20]:
print(notes_icd.loc[notes_icd.ICD9_CODE_x.isna()].HADM_ID.count())
print(notes_icd.loc[notes_icd.ICD9_CODE_x.isna()].HADM_ID.nunique())

6
4


In [21]:
admid_withnotes_withoutICD=notes_icd.loc[notes_icd.ICD9_CODE_x.isna()].HADM_ID.to_list()

In [22]:
len(admid_withnotes_withoutICD)

6

In [23]:
y=[b for b in admid_withnotes_withoutICD if b in admID_without_ICD9codes]

>AdmIds that do not have corresponding ICD codes in the discharge summary.

In [24]:
print(y)
print(admid_withnotes_withoutICD)

[110220.0, 110220.0, 182252.0, 109963.0, 142890.0, 109963.0]
[110220.0, 110220.0, 182252.0, 109963.0, 142890.0, 109963.0]


In [25]:
notes_icd.loc[notes_icd.duplicated(['HADM_ID'])]

,SUBJECT_ID,HADM_ID,CATEGORY,DESCRIPTION,ISERROR,TEXT,text_mod,sec,split,service:,social history:,family history:,past medical history:,chief complaint:,discharge medications:,discharge disposition:,discharge diagnosis:,allergies:,physical exam:,ICD9_CODE_x,ICD10,ICD9_CODE_y,icd10_pcs
13,5350,169684.0,Discharge summary,Report,NaN,Admission Date: [**2143-4-25**] Discharge...,admission date: <date> discharge date: ...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: cardiac surgery ', None, None,...",service: cardiac surgery,NaN,NaN,past medical history: 1. known coronary art...,"chief complaint: chest pain, 3-vessel diseas...",NaN,NaN,NaN,allergies: no known drug allergies. hospit...,NaN,"41401,4111,4019,53081,3659,3051","I2510,I200,I10,K219,Misc,F17200","3615,3612,3961","Misc,Misc,5A1.221Z"
21,9805,177212.0,Discharge summary,Report,NaN,Admission Date: [**2131-7-5**] D...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: surgery ', ' social history: m...",service: surgery,social history: married for 46 years and liv...,family history: father died at 62 from mi m...,"past medical history: 1. crohn's dz, found i...",chief complaint: new diagnosis colon cancer ...,discharge medications: 1. atenolol 50 mg tab...,discharge disposition: extended care faci...,discharge diagnosis: primary diagnosis: colo...,allergies: penicillins attending:<name>,physical exam: t-96.8 bp-140/70 p-96 rr-20 s...,"5781,42731,4254,4254,1536,5849,27652,5551,V125...","Misc,I4891,I425,I428,Misc,N179,E861,Misc,Z8671...","4525,9904,9907,4824","Misc,Misc,Misc,Misc"
35,710,114242.0,Discharge summary,Report,NaN,Admission Date: [**2182-2-18**] Dischar...,admission date: <date> discharge date...,"['Admission Date:', 'Discharge Date:', 'Birth:...",[' service: history of present illness: ...,service: history of present illness: th...,NaN,NaN,past medical history: hypertension. glauc...,NaN,discharge medications: none. follow up pla...,NaN,NaN,allergies: no known drug allergies. medic...,NaN,"0389,78559,4275,5185,570,5849,4280,5163,2875","A419,Misc,I469,Misc,Misc,N179,I509,Misc,D696","9604,9604,966,966,966,3778,3778,8964,3891,9390...","0BH.17EZ,0BH.18EZ,0DH.67UZ,0DH.68UZ,3E0.G36Z,5..."
55,4127,167565.0,Discharge summary,Report,NaN,Admission Date: [**2193-6-9**] Discharg...,admission date: <date> discharge date...,"['Admission Date:', 'Discharge Date:', 'Birth:...",[' service: <hospital> firm history of p...,service: <hospital> firm history of pre...,NaN,family history: no seizure disorder.,past medical history: seizure disorder secon...,NaN,NaN,NaN,NaN,"allergies: phenobarbital, penicillin, haldol....",NaN,"51881,51881,49392,5070,78039,319","J9600,J9690,Misc,J690,R569,Misc","9671,9671,9604,9604","5A1.935Z,5A1.945Z,0BH.17EZ,0BH.18EZ"
60,5239,129387.0,Discharge summary,Report,NaN,Admission Date: [**2189-2-18**] Dischar...,admission date: <date> discharge date...,"['Admission Date:', 'Discharge Date:', 'Servic...","["" service: ccu please note that this inte...",service: ccu please note that this interv...,NaN,NaN,NaN,NaN,discharge medications: lasix 80 mg by mouth...,NaN,discharge diagnosis: 1. status post myocardi...,NaN,NaN,"41011,78551,4280,5780,2830,51881,51881,4168,41...","Misc,Misc,I509,Misc,Misc,J9600,J9690,I272,I278...","3601,3606,3723,3723,3761,3761,8853,8856,9604,9...","Misc,Misc,4A0.20N8,4A0.23N8,5A0.2110,5A0.2210,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59647,93924,135672.0,Discharge summary,Addendum,NaN,"Name: [**Known lastname 18321**],[**Known fir...","name: <name>,<name> unit no:...","['Name:', 'Unit No:', 'Admission Date:', 'Disc...","[' service: medicine ', ' social history: ...",service: medicine,social history: patient is married with 3 gr...,family history: <name> son was diagnosed wit...,"past medical history: # diabetes mellitus, t...","chief complaint: small bowel perforation, ga...",discharge medications: 1. oxycodone-aceta

In [26]:
notes_icd.loc[notes_icd.HADM_ID==173989]

,SUBJECT_ID,HADM_ID,CATEGORY,DESCRIPTION,ISERROR,TEXT,text_mod,sec,split,service:,social history:,family history:,past medical history:,chief complaint:,discharge medications:,discharge disposition:,discharge diagnosis:,allergies:,physical exam:,ICD9_CODE_x,ICD10,ICD9_CODE_y,icd10_pcs
2749,26483,173989.0,Discharge summary,Report,NaN,Admission Date: [**2160-3-4**] Discharg...,admission date: <date> discharge date...,"['Admission Date:', 'Discharge Date:', 'Birth:...",[' service: cardiothoracic surgery hospita...,service: cardiothoracic surgery hospital ...,NaN,NaN,NaN,NaN,discharge medications: the patient to be goi...,NaN,discharge diagnosis: status post coronary art...,NaN,NaN,"99662,03811,4280,4240,4240,42731,40391,41011,2...","Misc,Misc,I509,I340,I348,I4891,I120,Misc,Misc,...","3512,3512,3512,3512,3512,3723,3723,8856,3615,3...","027.G04Z,027.G0DZ,027.G0ZZ,02N.G0ZZ,02Q.G0ZZ,4..."
50910,26483,173989.0,Discharge summary,Addendum,NaN,"Name: [**Known lastname 1193**], [**Known fir...","name: <name>, <name> a ...","['Name:', 'Unit No:', 'Admission Date:', 'Disc...","["" service: on [**3-31**], rehabilitation s...","service: on [**3-31**], rehabilitation scr...",NaN,NaN,NaN,NaN,discharge medications: noted in the prior dis...,NaN,NaN,NaN,NaN,"99662,03811,4280,4240,4240,42731,40391,41011,2...","Misc,Misc,I509,I340,I348,I4891,I120,Misc,Misc,...","3512,3512,3512,3512,3512,3723,3723,8856,3615,3...","027.G04Z,027.G0DZ,027.G0ZZ,02N.G0ZZ,02Q.G0ZZ,4..."


In [27]:
notes_icd.loc[notes_icd.DESCRIPTION=='Addendum']

,SUBJECT_ID,HADM_ID,CATEGORY,DESCRIPTION,ISERROR,TEXT,text_mod,sec,split,service:,social history:,family history:,past medical history:,chief complaint:,discharge medications:,discharge disposition:,discharge diagnosis:,allergies:,physical exam:,ICD9_CODE_x,ICD10,ICD9_CODE_y,icd10_pcs
50908,18689,117162.0,Discharge summary,Addendum,NaN,"Name: [**Known lastname 2132**], [**Known fir...","name: <name>, <name> unit ...","['Name:', 'Unit No:', 'Admission Date:', 'Disc...",[' service: [**company 112**] addendum: p...,service: [**company 112**] addendum: ple...,NaN,NaN,NaN,NaN,discharge medications: 2. atenolol 25 mg p.o...,NaN,NaN,NaN,NaN,"78039,25020,4010,V4581,V4582","R569,Misc,Misc,Z951,Z9861",NaN,NaN
50909,48687,164079.0,Discharge summary,Addendum,NaN,"Name: [**Known lastname 2152**],[**Known firs...","name: <name>,<name> a unit no: ...","['Name:', 'Unit No:', 'Admission Date:', 'Disc...","[' service: medicine ', None, None, None, N...",service: medicine,NaN,NaN,NaN,NaN,NaN,discharge disposition: home with service f...,NaN,allergies: gemcitabine attending:<name> ...,NaN,"486,51881,51881,5849,5119,1985,1970,78830,2768...","J189,J9600,J9690,N179,J918,Misc,Misc,Misc,E876...","9671,9671,3891,9604,9604,3893,3324","5A1.935Z,5A1.945Z,Misc,0BH.17EZ,0BH.18EZ,Misc,..."
50910,26483,173989.0,Discharge summary,Addendum,NaN,"Name: [**Known lastname 1193**], [**Known fir...","name: <name>, <name> a ...","['Name:', 'Unit No:', 'Admission Date:', 'Disc...","["" service: on [**3-31**], rehabilitation s...","service: on [**3-31**], rehabilitation scr...",NaN,NaN,NaN,NaN,discharge medications: noted in the prior dis...,NaN,NaN,NaN,NaN,"99662,03811,4280,4240,4240,42731,40391,41011,2...","Misc,Misc,I509,I340,I348,I4891,I120,Misc,Misc,...","3512,3512,3512,3512,3512,3723,3723,8856,3615,3...","027.G04Z,027.G0DZ,027.G0ZZ,02N.G0ZZ,02Q.G0ZZ,4..."
50911,92277,173474.0,Discharge summary,Addendum,NaN,"Name: [**Known lastname 1048**],[**Known firs...","name: <name>,<name> unit no: ...","['Name:', 'Unit No:', 'Admission Date:', 'Disc...","[' service: medicine ', None, None, None, N...",service: medicine,NaN,NaN,NaN,NaN,NaN,discharge disposition: home with service f...,NaN,allergies: sulfa (sulfonamide antibiotics) /...,NaN,"42823,4275,5849,4271,4168,4168,2761,4254,4254,...","Misc,I469,N179,I472,I272,I2789,E871,I425,I428,...","51,3775,8872,9962,9929","Misc,Misc,Misc,5A2.204Z,Misc"
50912,3659,193782.0,Discharge summary,Addendum,NaN,"Name: [**Known lastname 1494**], [**Known fir...","name: <name>, <name> u...","['Name:', 'Unit No:', 'Admission Date:', 'Disc...",['service: <name> addendum to coronary art...,service: <name> addendum to coronary arter...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"41071,4280,42731,41401,25000,4019,412","I214,I509,I4891,I2510,E119,I10,I252","3613,3722,3722,3722,3722,3615,3961,8853,8856","Misc,4A0.20N7,4A0.23N7,4A0.20N7,4A0.23N7,Misc,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59647,93924,135672.0,Discharge summary,Addendum,NaN,"Name: [**Known lastname 18321**],[**Known fir...","name: <name>,<name> unit no:...","['Name:', 'Unit No:', 'Admission Date:', 'Disc...","[' service: medicine ', ' social history: ...",service: medicine,social history: patient is married with 3 gr...,family history: <name> son was diagnosed wit...,"past medical history: # diabetes mellitus, t...","chief complaint: small bowel perforation, ga...",discharge medications: 1. oxycodone-acetamin...,discharge disposition: home with service ...,discharge diagnosis: peritonitis secondary to...,allergies: gluten / gentamicin / zithromax ...,physical exam: gen: chronically ill appearin...,"0389,56983,0785,00869,56721,78552,20280,99662,...","A419,Misc,Misc,Misc,Misc,R6521,Misc,Misc,Misc,...","4562,3893,3893","Misc,Misc,Misc"
59648,17782,155710.0,Discharge summary,Addendum,NaN,"Name: [**Known lastname 2706**], [**Known fir...","name: <name>, <name> i. ...","['Name:', 'Unit No:', 'Admission Date:', 'Disc...","["" service: <na

In [28]:
notes_icd.loc[notes_icd.HADM_ID==169684]

,SUBJECT_ID,HADM_ID,CATEGORY,DESCRIPTION,ISERROR,TEXT,text_mod,sec,split,service:,social history:,family history:,past medical history:,chief complaint:,discharge medications:,discharge disposition:,discharge diagnosis:,allergies:,physical exam:,ICD9_CODE_x,ICD10,ICD9_CODE_y,icd10_pcs
12,5350,169684.0,Discharge summary,Report,NaN,Admission Date: [**2143-4-25**] Discharge...,admission date: <date> discharge date: ...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: cardiac surgery ', None, None,...",service: cardiac surgery,NaN,NaN,past medical history: 1. known coronary art...,"chief complaint: chest pain, 3-vessel diseas...",NaN,NaN,NaN,allergies: no known drug allergies. hospit...,NaN,"41401,4111,4019,53081,3659,3051","I2510,I200,I10,K219,Misc,F17200","3615,3612,3961","Misc,Misc,5A1.221Z"
13,5350,169684.0,Discharge summary,Report,NaN,Admission Date: [**2143-4-25**] Discharge...,admission date: <date> discharge date: ...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: cardiac surgery ', None, None,...",service: cardiac surgery,NaN,NaN,past medical history: 1. known coronary art...,"chief complaint: chest pain, 3-vessel diseas...",NaN,NaN,NaN,allergies: no known drug allergies. hospit...,NaN,"41401,4111,4019,53081,3659,3051","I2510,I200,I10,K219,Misc,F17200","3615,3612,3961","Misc,Misc,5A1.221Z"


In [29]:
notes_icd_fewcols=notes_icd.drop(['SUBJECT_ID','ISERROR'],axis=1)

In [30]:
notes_icd_fewcols.fillna('',inplace=True)

In [31]:
notes_icd_fewcols

,HADM_ID,CATEGORY,DESCRIPTION,TEXT,text_mod,sec,split,service:,social history:,family history:,past medical history:,chief complaint:,discharge medications:,discharge disposition:,discharge diagnosis:,allergies:,physical exam:,ICD9_CODE_x,ICD10,ICD9_CODE_y,icd10_pcs
0,167853.0,Discharge summary,Report,Admission Date: [**2151-7-16**] Dischar...,admission date: <date> discharge date...,"['Admission Date:', 'Discharge Date:', 'Servic...",['service: addendum: radiologic studies: ...,service: addendum: radiologic studies: ra...,,,,,,,,,,"01193,4254,4254,42731,2639,2762,5070,5119,2113","Misc,I425,I428,I4891,E46,E872,J690,J918,Misc","4542,3491,3491,3893,3891","Misc,0W9.93ZZ,0W9.B3ZZ,Misc,Misc"
1,107527.0,Discharge summary,Report,Admission Date: [**2118-6-2**] Discharg...,admission date: <date> discharge date...,"['Admission Date:', 'Discharge Date:', 'Birth:...",[' service: micu and then to <name> medicine ...,service: micu and then to <name> medicine ...,"social history: lives with her husband, dr. ...",family history: noncontributory.,past medical history: 1. copd. last pulmona...,,discharge medications: 1. levothyroxine 75 mc...,,,allergies: norvasc leads to lightheadedness ...,,"5191,49121,51881,51881,486,2761,2449,311","Misc,Misc,J9600,J9690,J189,E871,E039,F329","9390,9390,9390,966,966,966,3199,9671,9671,9604...","5A0.9357,5A0.9457,5A0.9557,0DH.67UZ,0DH.68UZ,3..."
2,167118.0,Discharge summary,Report,Admission Date: [**2119-5-4**] D...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Servic...","[' service: cardiothoracic ', ' social hist...",service: cardiothoracic,social history: social history: the patient ...,family history: family history: (+) fhx cad;...,past medical history: copd flare [**6-7**] s...,"chief complaint: 81 yo f smoker w/ copd, sev...",discharge medications: 1. albuterol sulfate ...,discharge disposition: extended care faci...,"discharge diagnosis: copd, coronary artery di...",allergies: amlodipine attending:<name>,physical exam: admit h+p general-lovely 81 ...,"5191,5185,496,2762,45340,5533","Misc,Misc,J449,E872,Misc,Misc","3179,311,9672,9605,9605,9605,9605,9605,9605,96...","Misc,Misc,5A1.955Z,0B7.17DZ,0B7.18DZ,0BH.07DZ,..."
3,196489.0,Discharge summary,Report,Admission Date: [**2124-7-21**] ...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Servic...","[' service: medicine ', ' social history: ...",service: medicine,social history: the patient is married and w...,family history: (+) fhx cad; father with an ...,past medical history: # copd flare fev1 40% ...,chief complaint: copd exacerbation/shortness...,discharge medications: 1. metoprolol succina...,discharge disposition: extended care faci...,discharge diagnosis: primary: 1. chronic obs...,allergies: amlodipine attending:<name>,physical exam: on admission vitals: t: bp: ...,"51884,5849,34830,49121,2760,4160,3594,5780,276...","Misc,N179,Misc,Misc,E870,Misc,Misc,Misc,E872,M...","9672,9604,9604,3891,3893,4513,966,966,966","5A1.955Z,0BH.17EZ,0BH.18EZ,Misc,Misc,0DJ.08ZZ,..."
4,135453.0,Discharge summary,Report,Admission Date: [**2162-3-3**] D...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: medicine ', ' social history: ...",service: medicine,social history: patient recently discharged ...,family history: non contributory,past medical history: coronary artery diseas...,chief complaint: mr. <name> was seen at <hos...,discharge medications: 1. bisacodyl 10 mg su...,discharge disposition: extended care faci...,discharge diagnosis: 1. cervical spondylosis ...,allergies: patient recorded as having no kno...,physical exam: phycial exam prior to surgery...,"80506,5070,42823,2930,4538,E882,4280,4011,V450...","Misc,J690,Misc,F05,Misc,Misc,I509,Misc,Misc,I4...","8102,8103,353,8163,9671,9671,9604,9604","Misc,Misc,Misc,Misc,5A1.935Z,5A1.945Z,0BH.17EZ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59647,135672.0,Discharge 

In [32]:
notes_icd_agg=notes_icd_fewcols.groupby('HADM_ID').agg(','.join).reset_index()

In [33]:
notes_icd_agg

,HADM_ID,CATEGORY,DESCRIPTION,TEXT,text_mod,sec,split,service:,social history:,family history:,past medical history:,chief complaint:,discharge medications:,discharge disposition:,discharge diagnosis:,allergies:,physical exam:,ICD9_CODE_x,ICD10,ICD9_CODE_y,icd10_pcs
0,100001.0,Discharge summary,Report,Admission Date: [**2117-9-11**] ...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: medicine ', ' social history: ...",service: medicine,social history: patient lives at home in [**...,family history: both parents have htn and t2...,past medical history: type 1 diabetes mellit...,"chief complaint: nausea, vomiting major...",discharge medications: 1. citalopram 20 mg t...,discharge disposition: home,discharge diagnosis: diabetic keotacidosis h...,allergies: levaquin attending:<name>,"physical exam: gen: awake, alert, and orient...","25013,3371,5849,5780,V5867,25063,5363,4580,250...","Misc,Misc,N179,Misc,Z794,Misc,Misc,Misc,Misc,I...",,
1,100003.0,Discharge summary,Report,Admission Date: [**2150-4-17**] ...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: medicine ', ' social history: ...",service: medicine,social history: he lives alone. he is drink...,family history: nc,past medical history: hcv cirrhosis (tx with...,chief complaint: coffee ground emesis maj...,discharge medications: 1. phenol 1.4 % aeros...,discharge disposition: home,discharge diagnosis: peptic ulcer gi bleed ...,allergies: patient recorded as having no kno...,physical exam: on admission: vs: t95.9 hr 7...,"53100,2851,07054,5715,5715,5715,45621,53789,40...","Misc,D62,B182,K740,K7460,K7469,Misc,Misc,I10,M...","4443,9607,9607,9904,3893","0W3.P8ZZ,0D9.670Z,0D9.680Z,Misc,Misc"
2,100006.0,"Discharge summary,Discharge summary","Report,Addendum",Admission Date: [**2108-4-6**] Discharg...,admission date: <date> discharge date...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: o med ', ' social history: so...","service: o med , service: addendum to ...","social history: socially, she lives with her...",family history: family history shows father d...,past medical history: 1. multiple myeloma d...,chief complaint: dyspnea. history of pres...,",discharge medications: 1. coumadin 4 mg po q...",",",",","allergies: no known drug allergies. ,",",","49320,51881,51881,486,20300,2761,7850,3090,V12...","Misc,J9600,J9690,J189,Misc,E871,Misc,Misc,Z867...","9390,9390,9390,9925,9925,9925,9925,9390,9390,9...","5A0.9357,5A0.9457,5A0.9557,3E0.3305,3E0.4305,X..."
3,100007.0,Discharge summary,Report,Admission Date: [**2145-3-31**] ...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: surgery ', ' social history: m...",service: surgery,"social history: married, lives with husband ...","family history: father: deceased, leukemia ...",past medical history: chronic pancreatitis ...,chief complaint: severe abdominal and back p...,discharge medications: 1. oxycodone-acetamin...,discharge disposition: home,discharge diagnosis: small bowel obstruction ...,allergies: penicillins / dilantin attendi...,physical exam: t: 97.9 hr: 79 bp: 153/60 rr...,"56081,5570,9973,486,4019","Misc,Misc,Misc,J189,I10","4562,5459","Misc,Misc"
4,100009.0,Discharge summary,Report,Admission Date: [**2162-5-16**] ...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: cardiothoracic ', ' social hist...",service: cardiothoracic,social history: race: caucasian last dental...,"family history: father died 50yo cirrhosis, ...","past medical history: cad-(ami <date> , <na...",chief complaint: angina major surgical or...,discharge medications: 1. fenofibrate 160 mg...,discharge disposition: home with service ...,discharge diagnosis: coronary artery disease ...,allergies: no known allergies / adverse drug...,physical exam: pulse: 58 resp: 16 o2 sat:...,"41401,99604,4142,25000,27800,V8535,4148,4111,V...","I2510,Misc,Misc

In [34]:
def split_string_list(x):
    return list(set(re.split(",",x)))

In [35]:
def categoryfromlist(x):
  for i in x:
    y=re.search("(\d{3}|[VE]\d{2})",i)
    if y:
      return list(set([re.split("(\d{3}|[VE]\d{2})",i)[1] for i in x]))
    else:
      return []

In [36]:
notes_icd_agg['ICD9_CODE_x']=notes_icd_agg.ICD9_CODE_x.apply(split_string_list)

In [37]:
notes_icd_agg['ICD9_CATEGORY']=notes_icd_agg.ICD9_CODE_x.apply(categoryfromlist)

In [38]:
notes_icd_agg.loc[notes_icd_agg.HADM_ID.isin(admid_withnotes_withoutICD)]

,HADM_ID,CATEGORY,DESCRIPTION,TEXT,text_mod,sec,split,service:,social history:,family history:,past medical history:,chief complaint:,discharge medications:,discharge disposition:,discharge diagnosis:,allergies:,physical exam:,ICD9_CODE_x,ICD10,ICD9_CODE_y,icd10_pcs,ICD9_CATEGORY
5308,109963.0,"Discharge summary,Discharge summary","Report,Addendum",Admission Date: [**2185-8-21**] ...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: medicine ', ' social history: ...","service: medicine , service: medicine",social history: lives at <hospital> 174 [**l...,family history: patient unable to elucidate....,past medical history: 1) osteoarthritis 2) ...,chief complaint: shortness of breath majo...,discharge medications: 1. acetaminophen 650 ...,discharge disposition: extended care faci...,discharge diagnosis: urinary tract infection ...,allergies: no known allergies / adverse drug...,physical exam: admission: general: intubate...,[],",",",",",",[]
5458,110220.0,"Discharge summary,Discharge summary","Report,Report",Admission Date: [**2180-12-14**] ...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: surgery ', ' social history: +...","service: surgery , service: surgery","social history: +etoh occasional , social...","family history: non-contributory , family...","past medical history: none , past medical...",chief complaint: 20yo m s/p fall from 3 stor...,",","discharge disposition: expired , discharg...","discharge diagnosis: traumatic fall, intracra...",allergies: patient recorded as having no kno...,"physical exam: vss afebrile intubated, sed...",[],",",",",",",[]
22586,142890.0,Discharge summary,Report,Admission Date: [**2172-10-7**] ...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: surgery ', None, None, None, ' ...",service: surgery,,,,chief complaint: 40 yo ejection <name> bib m...,,discharge disposition: expired,discharge diagnosis: pt passed away on <date...,allergies: no drug allergy information on fi...,,[],,,,[]
43369,182252.0,Discharge summary,Report,Admission Date: [**2125-2-9**] D...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: medicine ', ' social history: ...",service: medicine,"social history: former smoker, but stopped i...",family history: non-contributory,past medical history: bladder tumor chf (ef...,chief complaint: hematuria major surgical...,discharge medications: 1. atorvastatin 20 mg...,discharge disposition: home,discharge diagnosis: primay: mechanical mitr...,allergies: ace inhibitors attending:<name>,"physical exam: physical exam: t: 96.9, bp: ...",[],,,,[]


In [39]:
notes_icd_agg.HADM_ID.nunique()

52726

In [40]:
total_icd9_codes=[a for b in notes_icd_agg.ICD9_CODE_x.to_list() for a in b]

In [41]:
dict_count_icd9_codes=dict(Counter(total_icd9_codes))
dict_count_icd9_codes=dict(sorted(dict_count_icd9_codes.items(),key=lambda x:x[1],reverse=True))

Dictionary of counts of different ICD codes in the discharge table(not in the whole dataset)

In [42]:
dict_count_icd9_codes

{'4019': 20046,
 '4280': 12842,
 '42731': 12589,
 '41401': 12178,
 '5849': 8906,
 '25000': 8783,
 '2724': 8503,
 '51881': 7249,
 '5990': 6442,
 '53081': 6154,
 '2720': 5766,
 '2859': 5295,
 '2449': 4785,
 '486': 4732,
 '2851': 4499,
 '2762': 4358,
 '496': 4296,
 '99592': 3792,
 'V5861': 3697,
 '5070': 3592,
 '0389': 3580,
 '5859': 3367,
 '40390': 3350,
 '311': 3347,
 '3051': 3272,
 '412': 3203,
 '2875': 3002,
 '41071': 3001,
 '2761': 2985,
 'V4581': 2943,
 '4240': 2876,
 'V1582': 2741,
 '5119': 2693,
 'V4582': 2651,
 '40391': 2566,
 'V290': 2529,
 '4241': 2517,
 '78552': 2501,
 'V5867': 2497,
 '42789': 2396,
 '32723': 2328,
 '9971': 2313,
 '5845': 2223,
 '2760': 2221,
 '7742': 2183,
 '5180': 2137,
 'V053': 2119,
 '4168': 2118,
 '49390': 2113,
 '2767': 2111,
 '45829': 2094,
 '2749': 2033,
 '4589': 1992,
 '73300': 1909,
 '78039': 1879,
 '5856': 1868,
 '5185': 1778,
 '4271': 1767,
 '4254': 1666,
 '4111': 1649,
 'V1251': 1567,
 '30000': 1552,
 '3572': 1552,
 '99811': 1508,
 '27800': 1483,


In [43]:
len(dict_count_icd9_codes)

6919

In [44]:
total_icd9_category=[a for b in notes_icd_agg.ICD9_CATEGORY.to_list() for a in b]

In [45]:
dict_count_icd9_category=dict(Counter(total_icd9_category))
dict_count_icd9_category=dict(sorted(dict_count_icd9_category.items(),key=lambda x: x[1],reverse=True))

In [46]:
print(dict_count_icd9_category)

{'401': 20646, '427': 16774, '276': 14712, '272': 14212, '414': 14081, '250': 13818, '428': 13330, '518': 12997, '285': 12404, '584': 11147, 'V45': 8846, '599': 7199, '530': 7191, 'V58': 6998, '585': 6764, 'E87': 6483, '403': 6297, 'V10': 6204, '038': 6085, '995': 5480, '424': 5404, '410': 5301, '780': 5296, '244': 5101, '997': 5078, '785': 5048, '305': 5000, '998': 4948, '458': 4935, '486': 4732, 'V15': 4420, '041': 4399, '496': 4296, '996': 4251, '287': 3881, 'V12': 3782, '790': 3672, '507': 3608, 'E93': 3473, '493': 3400, '311': 3347, '511': 3218, '412': 3203, '707': 3117, '348': 3112, '765': 2959, 'E88': 2916, '571': 2865, '300': 2855, '733': 2749, '278': 2664, '416': 2636, 'V29': 2591, '774': 2522, '070': 2481, '578': 2382, '787': 2359, 'V30': 2349, '327': 2349, '770': 2297, '274': 2295, '482': 2197, '280': 2148, 'V49': 2142, 'V43': 2131, 'V05': 2126, '425': 2056, '303': 2024, 'E84': 1988, '286': 1961, '789': 1960, '788': 1952, '294': 1927, '600': 1901, '411': 1900, '440': 1816, '

In [47]:
len(dict_count_icd9_category)

941

<table>
<tr>
<th> Datasets</th>
<th> Number of admissionIds</th>
<th> %coverage</th>
</tr>
<tr>
<td>Top 10 ICD9 codes</td>
<td>40562</td>
<td>76.93%</td>
</tr>
<tr>
<td>Top 20 ICD9 codes</td>
<td>43958</td>
<td>83.37%</td>
</tr>
<tr>
<td>Top 50 ICD9 codes</td>
<td>49534</td>
<td>93.95%</td>
</tr>
<tr>
<td>Top 10 ICD9 categories</td>
<td>44419</td>
<td>84.24%</td>
</tr>
<tr>
<td>Top 20 ICD9 categories</td>
<td>46143</td>
<td>87.51%</td>
</tr>
<tr>
<td>Top 50 ICD9 categories</td>
<td>51034</td>
<td>96.79%</td>
</tr>
</table>
This table shows how many average ICD9 codes are categories are there for each admids 
<table>
<tr><th>Type of ICD</th><th>Mean</th><th>Median</th></tr>
<tr><td>ICD9 codes</td><td>11.74</td><td>10</td></tr>
<tr><td>ICD9 categories</td><td>10.79</td><td>9</td></tr>
</table>

In [48]:
top10_icd9codes=list(dict_count_icd9_codes.keys())[:10]
top20_icd9codes=list(dict_count_icd9_codes.keys())[:20]
top50_icd9codes=list(dict_count_icd9_codes.keys())[:50]

In [49]:
notes_icd_agg.loc[notes_icd_agg.ICD9_CODE_x.apply(lambda x:any(item in top10_icd9codes for item in x))].HADM_ID.count()

40562

In [50]:
notes_icd_agg.loc[notes_icd_agg.ICD9_CODE_x.apply(lambda x:any(item in top20_icd9codes for item in x))].HADM_ID.count()

43958

In [51]:
notes_icd_agg.loc[notes_icd_agg.ICD9_CODE_x.apply(lambda x:any(item in top50_icd9codes for item in x))].HADM_ID.count()

49354

In [52]:
top10_icd9category=list(dict_count_icd9_category.keys())[:10]
top20_icd9category=list(dict_count_icd9_category.keys())[:20]
top50_icd9category=list(dict_count_icd9_category.keys())[:50]

In [53]:
notes_icd_agg.loc[notes_icd_agg.ICD9_CATEGORY.apply(lambda x:any(item in top10_icd9category for item in x))].HADM_ID.count()

44419

In [54]:
notes_icd_agg.loc[notes_icd_agg.ICD9_CATEGORY.apply(lambda x:any(item in top20_icd9category for item in x))].HADM_ID.count()

46143

In [55]:
notes_icd_agg.loc[notes_icd_agg.ICD9_CATEGORY.apply(lambda x:any(item in top50_icd9category for item in x))].HADM_ID.count()

51034

In [56]:
notes_icd_agg.ICD9_CODE_x.apply(lambda x:len(set(x))).median()
notes_icd_agg.ICD9_CODE_x.apply(lambda x:len(set(x))).mean()

11.738914387588666

In [57]:
notes_icd_agg.ICD9_CATEGORY.apply(lambda x:len(set(x))).median()
notes_icd_agg.ICD9_CATEGORY.apply(lambda x:len(set(x))).mean()

10.788870765846072

In [58]:
notes_icd_agg.loc[notes_icd_agg.ICD9_CATEGORY.apply(lambda x:len(x)>10)]

,HADM_ID,CATEGORY,DESCRIPTION,TEXT,text_mod,sec,split,service:,social history:,family history:,past medical history:,chief complaint:,discharge medications:,discharge disposition:,discharge diagnosis:,allergies:,physical exam:,ICD9_CODE_x,ICD10,ICD9_CODE_y,icd10_pcs,ICD9_CATEGORY
0,100001.0,Discharge summary,Report,Admission Date: [**2117-9-11**] ...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: medicine ', ' social history: ...",service: medicine,social history: patient lives at home in [**...,family history: both parents have htn and t2...,past medical history: type 1 diabetes mellit...,"chief complaint: nausea, vomiting major...",discharge medications: 1. citalopram 20 mg t...,discharge disposition: home,discharge diagnosis: diabetic keotacidosis h...,allergies: levaquin attending:<name>,"physical exam: gen: awake, alert, and orient...","[V5867, 5363, 25013, 5849, 7078, 5853, 25063, ...","Misc,Misc,N179,Misc,Z794,Misc,Misc,Misc,Misc,I...",,,"[458, 337, V58, 707, 403, 584, 250, V13, 578, ..."
4,100009.0,Discharge summary,Report,Admission Date: [**2162-5-16**] ...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: cardiothoracic ', ' social hist...",service: cardiothoracic,social history: race: caucasian last dental...,"family history: father died 50yo cirrhosis, ...","past medical history: cad-(ami <date> , <na...",chief complaint: angina major surgical or...,discharge medications: 1. fenofibrate 160 mg...,discharge disposition: home with service ...,discharge diagnosis: coronary artery disease ...,allergies: no known allergies / adverse drug...,physical exam: pulse: 58 resp: 16 o2 sat:...,"[V5867, 4262, 2720, 4019, V153, V4582, 2859, V...","I2510,Misc,Misc,E119,E669,Misc,Misc,I200,Z9861...","3613,3615,3795,3961","Misc,Misc,Misc,5A1.221Z","[278, V45, 411, V58, 996, 414, 272, 250, V15, ..."
6,100011.0,Discharge summary,Report,Admission Date: [**2177-8-29**] ...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: surgery ', ' social history: t...",service: surgery,social history: tobacco none etoh none,family history: non-contributory.,past medical history: none,chief complaint: helmeted motocyclist who hi...,discharge medications: 1. acetaminophen 325 ...,discharge disposition: extended care faci...,discharge diagnosis: s/p scooter v tree 1. l...,allergies: patient recorded as having no kno...,physical exam: 96.9 130 150/97 20 100% ...,"[86403, 8052, E8152, 8220, 48242, 82111, 8900,...","Misc,Misc,Misc,Misc,Misc,D62,Misc,Misc,Misc,Mi...","7965,311,387,7936,7915,110,9672,9604,9604,3323...","Misc,Misc,Misc,Misc,Misc,Misc,5A1.955Z,0BH.17E...","[860, 482, 864, 852, E81, 805, 807, 821, 890, ..."
8,100016.0,Discharge summary,Report,Admission Date: [**2188-5-24**] ...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: medicine ', ' social history: ...",service: medicine,"social history: lives at group home, but has...",family history: nc,past medical history: - downs syndrome - mr...,chief complaint: respiratory failure majo...,,discharge disposition: extended care faci...,discharge diagnosis: primary diagnosis: subg...,allergies: ampicillin / thorazine attendi...,physical exam: admission exam: gen: comfort...,"[47874, 45829, 30000, 51881, 25541, 7580, 3459...","J690,J9600,J9690,Misc,Misc,Misc,Misc,Misc,I952...","311,9672,3323,3323,3323,966,966,966,3897","Misc,5A1.955Z,0BJ.08ZZ,0BJ.K8ZZ,0BJ.L8ZZ,0DH.6...","[458, 300, 255, 518, 758, 345, 507, V44, 478, ..."
9,100018.0,Discharge summary,Report,Admission Date: [**2176-8-29**] ...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: medicine ', ' social history: ...",service: medicine,social history: the patient is married and h...,family history: heart disease in unspecifice...,past medical history: (1) type 2 diabetes me...,chief complaint: upper 

Procedure codes in the whole MIMIC dataset. There are 52243 AdmIDs are there for which there are 2009 ICD9 PCS codes.<br> In the discharge summary which has 52726 admids there are ICD9 PCS codes for only 46841 (5885 do not have any codes corresponding to them). There are total 1989 unique ICD9 PCS codes in the discharge summary.

In [59]:
allmimic_procedure=pd.read_csv('/content/drive/My Drive/AlgoIntern/Data/PROCEDURES_ICD.csv')

In [60]:
allmimic_procedure.HADM_ID.nunique()

52243

In [61]:
adm_icdpcs=pd.read_csv('/content/drive/My Drive/AlgoIntern/Data/Data1.0/adm_icdpcs.csv')

In [62]:
adm_icdpcs

,HADM_ID,ICD9_CODE,icd10_pcs
0,100003,"4443,9607,9607,9904,3893","0W3.P8ZZ,0D9.670Z,0D9.680Z,Misc,Misc"
1,100006,"9390,9390,9390,9925,9925,9925,9925","5A0.9357,5A0.9457,5A0.9557,3E0.3305,3E0.4305,X..."
2,100007,"4562,5459","Misc,Misc"
3,100009,"3613,3615,3795,3961","Misc,Misc,Misc,5A1.221Z"
4,100010,"5551,540,403","Misc,Misc,Misc"
...,...,...,...
52238,199993,"3614,3512,3512,3512,3512,3512,3761,3761,8842,8...","Misc,027.G04Z,027.G0DZ,027.G0ZZ,02N.G0ZZ,02Q.G..."
52239,199994,"9671,9671,9604,9604,3995,3995,3891","5A1.935Z,5A1.945Z,0BH.17EZ,0BH.18EZ,5A1.D00Z,5..."
52240,199995,"3521,3961,3845,3539,8841,8847,9929,8872,3699","Misc,5A1.221Z,Misc,Misc,Misc,Misc,Misc,Misc,Misc"
52241,199998,"3612,3615,3964","Misc,Misc,Misc"


In [63]:
allmimic_procedure.ICD9_CODE.nunique()

2009

In [64]:
len(set([a for b in adm_icdpcs.ICD9_CODE.apply(split_string_list).to_list() for a in b]))

2009

In [65]:
notes_icd

,SUBJECT_ID,HADM_ID,CATEGORY,DESCRIPTION,ISERROR,TEXT,text_mod,sec,split,service:,social history:,family history:,past medical history:,chief complaint:,discharge medications:,discharge disposition:,discharge diagnosis:,allergies:,physical exam:,ICD9_CODE_x,ICD10,ICD9_CODE_y,icd10_pcs
0,22532,167853.0,Discharge summary,Report,NaN,Admission Date: [**2151-7-16**] Dischar...,admission date: <date> discharge date...,"['Admission Date:', 'Discharge Date:', 'Servic...",['service: addendum: radiologic studies: ...,service: addendum: radiologic studies: ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"01193,4254,4254,42731,2639,2762,5070,5119,2113","Misc,I425,I428,I4891,E46,E872,J690,J918,Misc","4542,3491,3491,3893,3891","Misc,0W9.93ZZ,0W9.B3ZZ,Misc,Misc"
1,13702,107527.0,Discharge summary,Report,NaN,Admission Date: [**2118-6-2**] Discharg...,admission date: <date> discharge date...,"['Admission Date:', 'Discharge Date:', 'Birth:...",[' service: micu and then to <name> medicine ...,service: micu and then to <name> medicine ...,"social history: lives with her husband, dr. ...",family history: noncontributory.,past medical history: 1. copd. last pulmona...,NaN,discharge medications: 1. levothyroxine 75 mc...,NaN,NaN,allergies: norvasc leads to lightheadedness ...,NaN,"5191,49121,51881,51881,486,2761,2449,311","Misc,Misc,J9600,J9690,J189,E871,E039,F329","9390,9390,9390,966,966,966,3199,9671,9671,9604...","5A0.9357,5A0.9457,5A0.9557,0DH.67UZ,0DH.68UZ,3..."
2,13702,167118.0,Discharge summary,Report,NaN,Admission Date: [**2119-5-4**] D...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Servic...","[' service: cardiothoracic ', ' social hist...",service: cardiothoracic,social history: social history: the patient ...,family history: family history: (+) fhx cad;...,past medical history: copd flare [**6-7**] s...,"chief complaint: 81 yo f smoker w/ copd, sev...",discharge medications: 1. albuterol sulfate ...,discharge disposition: extended care faci...,"discharge diagnosis: copd, coronary artery di...",allergies: amlodipine attending:<name>,physical exam: admit h+p general-lovely 81 ...,"5191,5185,496,2762,45340,5533","Misc,Misc,J449,E872,Misc,Misc","3179,311,9672,9605,9605,9605,9605,9605,9605,96...","Misc,Misc,5A1.955Z,0B7.17DZ,0B7.18DZ,0BH.07DZ,..."
3,13702,196489.0,Discharge summary,Report,NaN,Admission Date: [**2124-7-21**] ...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Servic...","[' service: medicine ', ' social history: ...",service: medicine,social history: the patient is married and w...,family history: (+) fhx cad; father with an ...,past medical history: # copd flare fev1 40% ...,chief complaint: copd exacerbation/shortness...,discharge medications: 1. metoprolol succina...,discharge disposition: extended care faci...,discharge diagnosis: primary: 1. chronic obs...,allergies: amlodipine attending:<name>,physical exam: on admission vitals: t: bp: ...,"51884,5849,34830,49121,2760,4160,3594,5780,276...","Misc,N179,Misc,Misc,E870,Misc,Misc,Misc,E872,M...","9672,9604,9604,3891,3893,4513,966,966,966","5A1.955Z,0BH.17EZ,0BH.18EZ,Misc,Misc,0DJ.08ZZ,..."
4,26880,135453.0,Discharge summary,Report,NaN,Admission Date: [**2162-3-3**] D...,admission date: <date> dischar...,"['Admission Date:', 'Discharge Date:', 'Birth:...","[' service: medicine ', ' social history: ...",service: medicine,social history: patient recently discharged ...,family history: non contributory,past medical history: coronary artery diseas...,chief complaint: mr. <name> was seen at <hos...,discharge medications: 1. bisacodyl 10 mg su...,discharge disposition: extended care faci...,discharge diagnosis: 1. cervical spondylosis ...,allergies: patient recorded as having no kno...,physical exam: phycial exam prior to surgery...,"80506,5070,42823,2930,4538,E882,4280,4011,V450...","Misc,J690,Misc,F05,Misc,Misc,I509,Misc,Misc,I4...","8102,8103,353,8163,9671,9671,9604,9604","Misc,Misc,Misc,Misc,5A1.935Z,5A1.945Z,0BH.17EZ..."
...,.

In [66]:
notes_icd.loc[notes_icd.ICD9_CODE_y.isna()].HADM_ID.count()

6284

In [67]:
notes_icd.loc[notes_icd.ICD9_CODE_y.isna()].HADM_ID.nunique()

5885

In [68]:
notes_icd_agg['ICD9_CODE_y']=notes_icd_agg.ICD9_CODE_y.apply(split_string_list)

In [69]:
total_icd9pcscodes=[a for b in notes_icd_agg.ICD9_CODE_y.to_list() for a in b if a != '']

In [70]:
dict_count_icd9pcscodes=dict(Counter(total_icd9pcscodes))
dict_count_icd9pcscodes=dict(sorted(dict_count_icd9pcscodes.items(),key=lambda x:x[1],reverse=True))

In [71]:
len(set(total_icd9pcscodes))

1989

In [72]:
len(dict_count_icd9pcscodes)

1989

Mean and median number of ICD9 PCS codes for each AdmIDs in the discharge summary table.

In [73]:
notes_icd_agg.ICD9_CODE_y.apply(lambda x:len(set(x))).mean()

4.25439062322194

In [74]:
notes_icd_agg.ICD9_CODE_y.apply(lambda x:len(set(x))).median()

3.0

In [75]:
print(dict_count_icd9pcscodes)

{'3893': 12866, '9604': 9493, '966': 9102, '9671': 8426, '9904': 7102, '3961': 6781, '9672': 5815, '8856': 5045, '3891': 4449, '3615': 4387, '9915': 4162, '8872': 3305, '3722': 3147, '3995': 3133, '9390': 2656, '3723': 2619, '3324': 2607, '4513': 2552, '9955': 2275, '9983': 2183, '8853': 2066, '331': 1999, '9907': 1987, '3612': 1897, '311': 1752, '9920': 1636, '3491': 1590, '3404': 1545, '4311': 1519, '40': 1502, '3613': 1497, '3895': 1493, '3897': 1462, '5491': 1420, '3521': 1415, '8841': 1243, '66': 1226, '9905': 1211, '3606': 1166, '3607': 1100, '8964': 1049, '3322': 999, '3323': 994, '3601': 994, '3761': 986, '3611': 945, '45': 847, '17': 818, '8855': 808, '640': 805, '9962': 760, '387': 750, '4523': 744, '14': 732, '8842': 723, '5459': 675, '8847': 667, '9910': 654, '3950': 650, '8659': 647, '159': 604, '4443': 590, '9960': 588, '4516': 578, '8848': 561, '9605': 558, '3721': 551, '9607': 538, '8622': 538, '8852': 522, '3845': 517, '3522': 506, '9925': 498, '5187': 472, '131': 471,

<table>
<tr>
<th>type of dataset</th><th>Number of Adm ids</th><th>% coverage</th>
</tr>
<tr>
<td>Top10 ICD9 PCS codes</td><td>33304</td><td>71.10%</td>
</tr>
<tr>
<td>Top20 ICD9 PCS codes</td><td>37931</td><td>80.98%</td>
</tr>
<tr>
<td>Top50 ICD9 PCS codes</td><td>40493</td><td>86.45%</td>
</tr>
</table>

In [76]:
top10_icd9pcscodes=list(dict_count_icd9pcscodes.keys())[:10]
top20_icd9pcscodes=list(dict_count_icd9pcscodes.keys())[:20]
top50_icd9pcscodes=list(dict_count_icd9pcscodes.keys())[:50]

In [77]:
top10_icd9pcscodes

['3893', '9604', '966', '9671', '9904', '3961', '9672', '8856', '3891', '3615']

In [78]:
notes_icd_agg.loc[notes_icd_agg.ICD9_CODE_y.apply(lambda x:any(item in top10_icd9pcscodes for item in x))].HADM_ID.nunique()

33304

In [79]:
notes_icd_agg.loc[notes_icd_agg.ICD9_CODE_y.apply(lambda x:any(item in top20_icd9pcscodes for item in x))].HADM_ID.count()

37931

In [80]:
notes_icd_agg.loc[notes_icd_agg.ICD9_CODE_y.apply(lambda x:any(item in top50_icd9pcscodes for item in x))].HADM_ID.nunique()

40493

Below code snippet is for extracting file from .gz compressed folders.

In [81]:
#import gzip
#import shutil
#with gzip.open('/content/drive/My Drive/AlgoIntern/Data/NOTEEVENTS.csv.gz', 'rb') as f_in:
#    with open('/content/drive/My Drive/AlgoIntern/Data/NOTEEVENTS.csv', 'wb') as f_out:
#        shutil.copyfileobj(f_in, f_out)

In [82]:
note_events=pd.read_csv('/content/drive/My Drive/AlgoIntern/Data/NOTEEVENTS.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [83]:
note_events.loc[note_events.CATEGORY=='Discharge summary'].HADM_ID.nunique()

52726

In [84]:
different_note_category=note_events.CATEGORY.unique().tolist()

>Total AdmIDS in the 'NOTEEVENTS' table is 58361 and different number of admids corresponding to various categories are shown below.

In [85]:
note_events.HADM_ID.nunique()

58361

In [86]:
for i in different_note_category:
  print("No. of unique AdmID for category "+i+":",note_events.loc[note_events.CATEGORY==i].HADM_ID.nunique())

No. of unique AdmID for category Discharge summary: 52726
No. of unique AdmID for category Echo: 23585
No. of unique AdmID for category ECG: 44185
No. of unique AdmID for category Nursing: 9070
No. of unique AdmID for category Physician : 8983
No. of unique AdmID for category Rehab Services: 2249
No. of unique AdmID for category Case Management : 619
No. of unique AdmID for category Respiratory : 3986
No. of unique AdmID for category Nutrition: 3167
No. of unique AdmID for category General: 3170
No. of unique AdmID for category Social Work: 1393
No. of unique AdmID for category Pharmacy: 68
No. of unique AdmID for category Consult: 50
No. of unique AdmID for category Radiology: 45526
No. of unique AdmID for category Nursing/other: 34890


In [87]:
print (different_note_category)

['Discharge summary', 'Echo', 'ECG', 'Nursing', 'Physician ', 'Rehab Services', 'Case Management ', 'Respiratory ', 'Nutrition', 'General', 'Social Work', 'Pharmacy', 'Consult', 'Radiology', 'Nursing/other']


In [88]:
category_hadmids={'category':[],'Number_of_admissions':[]}
for i in different_note_category:
  # print("No. of unique AdmID for category "+i+":",note_events.loc[note_events.CATEGORY==i].HADM_ID.nunique())
  category_hadmids['category'].append(i)
  category_hadmids['Number_of_admissions'].append(note_events.loc[note_events.CATEGORY==i].HADM_ID.nunique())

In [89]:
pd.DataFrame.from_dict(category_hadmids)

,category,Number_of_admissions
0,Discharge summary,52726
1,Echo,23585
2,ECG,44185
3,Nursing,9070
4,Physician,8983
5,Rehab Services,2249
6,Case Management,619
7,Respiratory,3986
8,Nutrition,3167
9,General,3170
